# Import Libraries

In [1]:
import numpy as np
import datetime as dt
from datetime import date
from datetime import datetime,timedelta
import pandas as pd
import pandas_datareader.data as web
import math
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
yf.pdr_override()
from scipy.stats import norm
N = norm.cdf


days_year=365

     |████████████████████████████████| 6.3MB 7.8MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=d2449200f11cb83e67e0f82043c3ec4649f1c5e0489d1d301f1d50a319a7a7d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-eua55d00/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


# Getting The Option Information

In [2]:
alfabeto=["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q",'R',"S","T","U","V","W","X","Y","Z"]
delta, vega,theta,gamma,rho,episilon,zomma,charm,veta,vanna,volta=0,0,0,0,0,0,0,0,0,0,0
code_list=[]
quantidade_list=[]
custo=[]
while True:
  code=str(input("Code: "))
  if code=="-":
    break
  quantidade=float(input("Quantity: "))
  print("-="*20)
  code_list.append(code)
  quantidade_list.append(quantidade)
code=code_list[0]
def information (code):
  contador=0
  while True:
    if code[contador] not in alfabeto:
      ticker=code[:contador]
      break
    contador+=1
  expiration_year=int("20"+code[contador:contador+2])
  expiration_month=int(code[contador+2:contador+4])
  expiration_day=int(code[contador+4:contador+6])
  today = date.today()
  expiration=date(expiration_year,expiration_month,expiration_day)
  DTE=expiration-today
  T=(DTE.days)/days_year  
  tipo=code[contador+6]
  S=code[contador+7:contador+14]
  # pegando valores do mercado
  ticker = yf.Ticker(ticker)
  todays_data = ticker.history(period='1d')
  code=yf.Ticker(code)
  price_option=(code.history(period="2d"))["Close"][0]
  custo.append(price_option*-quantidade)
  return ticker,tipo,S,T,todays_data["Close"][0],price_option
# setando os paramêtros da opção 1
ticker=information(code)[0]
tipo=information(code)[1]
K=float(information(code)[2])/100
T=information(code)[3]
S=information(code)[4]
price_option=information(code)[5]
quantidade=quantidade_list[0]



# exemplos de codigos
#AMZN210604C03200000
#AMZN210604C03215000

Code: AMZN210604C03200000
Quantity: 1
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Code: AMZN210604C03215000
Quantity: -1
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Code: -


# Setting Risk Free Rate & Dividends

In [3]:
r=0.02
q=0.0

# Black Scholes Formula

In [4]:
def d1_func(S,K,r,q,sigma,T):
  d1=(np.log(S/K)+(r-q+sigma**2/2)*T)/(sigma*math.sqrt(T))
  return d1
def d2_func(S,K,r,q,sigma,T):
   d1=(np.log(S/K)+(r-q+sigma**2/2)*T)/(sigma*math.sqrt(T))
   d2=d1-sigma*math.sqrt(T)
   return d2
def black_scholes_call(S,K,r,q,sigma,T):
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  price=S*math.exp(-q*T)*N(d1)-math.exp(-r*T)*K*N(d2)
  return price
def black_scholes_put(S,K,r,q,sigma,T):
    d1=d1_func(S,K,r,q,sigma,T)
    d2=d2_func(S,K,r,q,sigma,T)
    price=math.exp(-r*T)*K*N(-d2)-S*math.exp(-q*T)*N(-d1)
    return price

def phi_d1_func(d1):
  d1=d1_func(S,K,r,q,sigma,T)
  phid1=(math.exp((-d1**2)/2)/math.sqrt(2*math.pi))
  return phid1
def phi_d2_func(d2):
  d2=d2_func(S,K,r,q,sigma,T)
  phid2=math.exp((-d2**2)/2)/math.sqrt(2*math.pi)
  return phid2

# Calculating the Implied Volatility

In [5]:
def IV(price_option,S,K,T,r):
  precision=0.00001
  upper_vol=50.0
  max_vol=50.0
  min_vol=0.0001
  lower_vol=0.0001
  iteration=0
  while 1:
    iteration+=1
    mid_vol=(upper_vol+lower_vol)/2.0
    if tipo=="C":
      preco=black_scholes_call(S,K,r,q,mid_vol,T)
      lower_price=black_scholes_call(S,K,r,q,lower_vol,T)
      if (lower_price-price_option)*(preco-price_option)>0:
        lower_vol=mid_vol
      else:
        upper_vol=mid_vol
      if abs(preco-price_option)<precision:break
      if mid_vol>max_vol-5:
        mid_vol-0.0001
        break
    elif tipo=="P":
      preco=black_scholes_put(S,K,r,q,mid_vol,T)
      upper_price=black_scholes_put(S,K,r,q,upper_vol,T)
      if (upper_price-price_option)*(preco-price_option)>0:
        upper_vol=mid_vol
      else:
        lower_vol=mid_vol
      if abs(preco-price_option)<precision:break
      if iteration>100:break
  return mid_vol
sigma=IV(price_option,S,K,T,r)







# 1° Order Greeks

In [6]:
def delta_func(S,K,r,q,sigma,T,tipo,quantidade):
  d1=d1_func(S,K,r,q,sigma,T)
  if tipo=="C":
    delta=math.exp(-q*T)*N(d1)*quantidade
  if tipo=="P":
    delta=-math.exp(-q*T)*N(-d1)*quantidade
  return delta
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def vega_func(S,K,r,q,sigma,T,tipo,quantidade):
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid1=phi_d1_func(d1)
  vega=S*math.exp(-q*T)*phid1*math.sqrt(T)/100*quantidade
  return vega
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def theta_func(S,K,r,q,sigma,T,tipo,days_year,quantidade):
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid1=phi_d1_func(d1)
  if tipo=="C":
    theta=((-math.exp(-q*T)*((S*phid1*sigma)/(2*math.sqrt(T)))-r*K*math.exp(-r*T)*N(d2)+q*S*math.exp(-q*T)*N(d1))/days_year)*quantidade
  if tipo =="P":
    theta=((-math.exp(-q*T)*((S*phid1*sigma)/(2*math.sqrt(T)))+r*K*math.exp(-r*T)*N(-d2)-q*S*math.exp(-q*T)*N(-d1))/days_year)*quantidade
  return theta
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def rho_func(S,K,r,q,sigma,T,tipo,quantidade):
  d2=d2_func(S,K,r,q,sigma,T)
  if tipo=="C":
    rho=(K*T*math.exp(-r*T)*N(d2)/100)*quantidade
  if tipo=="P":
    rho=(-K*T*math.exp(-r*T)*N(-d2)/100)*quantidade
  return rho
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def episilon_func(S,K,r,q,sigma,T,tipo,quantidade): # Percentage change in option value per percentage change in the underlying dividend yield
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  if tipo=="C":
    episilon=(-S*T*math.exp(-q*T)*N(d2)/100)*quantidade
  if tipo =="P":
    episilon=(S*T*math.exp(-q*T)*N(-d1)/100)*quantidade
  return episilon
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Lambda greek :(

delta=delta_func(S,K,r,q,sigma,T,tipo,quantidade)
vega=vega_func(S,K,r,q,sigma,T,tipo,quantidade)
theta=theta_func(S,K,r,q,sigma,T,tipo,days_year,quantidade)
rho=rho_func(S,K,r,q,sigma,T,tipo,quantidade)
episilon=episilon_func(S,K,r,q,sigma,T,tipo,quantidade)





# 2° Order Greeks

In [7]:
def gamma_func(S,K,r,q,sigma,T,tipo,quantidade):
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid2=phi_d2_func(d2)
  gamma=K*math.exp(-r*T)*((phid2)/(S**2*sigma*math.sqrt(T)))*quantidade
  return gamma
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def vanna_func(S,K,r,q,sigma,T,tipo,quantidade): # Dvega/Dspot  --> mathematically equivalent to Ddelta/Dvol , helps to make a effective delta hedge as volatility changes or the effectivness of a vega hedge against change in Spot Price
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid1=phi_d1_func(d1)
  vanna=(-math.exp(-q*T)*phid1*(d2/sigma))*quantidade
  return vanna
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def charm_func(S,K,r,q,sigma,T,tipo,days_year,quantidade):# Rate of change of delta in time --> Ddelta/Dtime is an important indicator in a delta-hedging, divide per number of days to find delta decay per day, near to expiration charm changes quickly becoming innacurate
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid1=phi_d1_func(d1)
  if tipo=="C":
    charm=((q*math.exp(-q*T)*N(d1)-math.exp(-q*T)*phid1*((2*(r-q)*T-d2*sigma*math.sqrt(T))/(2*T*sigma*math.sqrt(T))))/days_year)*quantidade
  if tipo=="P":
    charm=((-q*math.exp(-q*T)*N(-d1)-math.exp(-q*T)*phid1*((2*(r-q)*T-d2*sigma*math.sqrt(T))/(2*T*sigma*math.sqrt(T))))/days_year)*quantidade
  return charm
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def vomma_func(S,K,r,q,sigma,T,tipo,quantidade):# vega convexity, the rate of change in vega with respect to volatility, positive vomma indicates that you are long vega and negative vomma you are short vega, if you have a position like neutral vega you need to understand vomma to analise what happens in vega if the iv changes, positive for away from the money
  # rate of change
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  vomma=((vega_func(S,K,r,q,sigma,T,tipo,quantidade)*(d1*d2/sigma)))*quantidade
  return vomma
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def veta_func(S,K,r,q,sigma,T,tipo,quantidade):# rate of change in vega by passage of time
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid1=phi_d1_func(d1)
  veta=(-S*math.exp(-q*T)*phid1*math.sqrt(T)*(q+((r-q)*d1)/(sigma*math.sqrt(T))-((1+d1*d2)/(2*T)))/((days_year)*100))*quantidade
  return veta
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


gamma=gamma_func(S,K,r,q,sigma,T,tipo,quantidade)
vanna=vanna_func(S,K,r,q,sigma,T,tipo,quantidade)
charm=charm_func(S,K,r,q,sigma,T,tipo,days_year,quantidade)# percentage
vomma=vomma_func(S,K,r,q,sigma,T,tipo,quantidade)# percentage
veta= veta_func(S,K,r,q,sigma,T,tipo,quantidade)# Percentage
# dividir pela soma das quantidades nas gregas de porcentagem

# 3° Order Greeks

In [8]:
def speed_func(S,K,r,q,sigma,T,tipo,quantidade):# Change in Gamma with respect to Spot Price, important to delta hedging or gamma-hedging
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid1=phi_d1_func(d1)
  speed=(-math.exp(-q*T)*(phid1/(S**2*sigma*math.sqrt(T)))*(d1/(sigma*math.sqrt(T))+1)*quantidade)
  return speed
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def zomma_func(S,K,r,q,sigma,T,tipo,quantidade):# rate of change of gamma with respect to volatility, gamma-hedging helps to antecipate hedge positions as volatility changes
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid1=phi_d1_func(d1)
  zomma=math.exp(-q*T)*((phid1*(d1*d2-1)/(S*sigma**2*math.sqrt(T))))*quantidade
  return zomma
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def color_func(S,K,r,q,sigma,T,tipo,quantidade):# rate of change in gamma with respect of passage of time, gamma hedge is expressed by gamma per year so we divide by number of days
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid1=phi_d1_func(d1)
  color=((-math.exp(-q*T)*((phid1)/(2*S*T*sigma*math.sqrt(T)))*(2*q*T+1+((2*(r-q)*T-d2*sigma*math.sqrt(T)))/sigma*math.sqrt(T)*d1))/days_year)*quantidade
  return color
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def ultima_func(S,K,r,q,sigma,T,tipo,quantidade):# Sensivity to vomma with respect to volatility
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  vega=vega_func(S,K,r,q,sigma,T,tipo,quantidade)
  ultima=(-vega/sigma**2)*(d1*d2*(1-d1*d2)+d1**2+d2**2)*quantidade
  return ultima
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def dual_delta_func(S,K,r,q,sigma,T,tipo,quantidade):
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  if tipo=="C":
    dual_delta=(-math.exp(-r*T)*N(d2))*quantidade
  if tipo=="P":
    dual_delta=(math.exp(-r*T)*N(-d2))*quantidade
  return dual_delta
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
def dual_gamma_func(S,K,r,q,sigma,T,tipo,quantidade):
  d1=d1_func(S,K,r,q,sigma,T)
  d2=d2_func(S,K,r,q,sigma,T)
  phid2=phi_d2_func(d2)
  dual_gamma=math.exp(-r*T)*(phid2/(K*sigma*math.sqrt(T)))*quantidade
  return dual_gamma
#-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=




speed=speed_func(S,K,r,q,sigma,T,tipo,quantidade)
zomma=zomma_func(S,K,r,q,sigma,T,tipo,quantidade)
color=color_func(S,K,r,q,sigma,T,tipo,quantidade)
ultima=ultima_func(S,K,r,q,sigma,T,tipo,quantidade)
dual_delta=dual_delta_func(S,K,r,q,sigma,T,tipo,quantidade)
dual_gamma=dual_gamma_func(S,K,r,q,sigma,T,tipo,quantidade)




# Computing others options

In [9]:
if len(code_list)>1:
  for a in range(len(code_list)-1):
    print(code_list[a+1])
    code=code_list[a+1]
    # setando as informações
    ticker=information(code)[0]
    tipo=information(code)[1]
    K=float(information(code)[2])/100
    T=information(code)[3]
    S=information(code)[4]
    price_option=information(code)[5]
    quantidade=quantidade_list[a+1]
    # chamando as funções
    delta+=delta_func(S,K,r,q,sigma,T,tipo,quantidade)
    rho+=rho_func(S,K,r,q,sigma,T,tipo,quantidade)
    theta+=theta_func(S,K,r,q,sigma,T,tipo,days_year,quantidade)
    vega+=vega_func(S,K,r,q,sigma,T,tipo,quantidade)
    gamma+=gamma_func(S,K,r,q,sigma,T,tipo,quantidade)
    vanna+=vanna_func(S,K,r,q,sigma,T,tipo,quantidade)
    charm+=charm_func(S,K,r,q,sigma,T,tipo,days_year,quantidade)# percentage
    vomma+=vomma_func(S,K,r,q,sigma,T,tipo,quantidade)# percentage
    veta+=veta_func(S,K,r,q,sigma,T,tipo,quantidade)# Percentage
    speed+=speed_func(S,K,r,q,sigma,T,tipo,quantidade)
    zomma+=zomma_func(S,K,r,q,sigma,T,tipo,quantidade)
    color+=color_func(S,K,r,q,sigma,T,tipo,quantidade)
    ultima+=ultima_func(S,K,r,q,sigma,T,tipo,quantidade)
    dual_delta+=dual_delta_func(S,K,r,q,sigma,T,tipo,quantidade)
    dual_gamma+=dual_gamma_func(S,K,r,q,sigma,T,tipo,quantidade)
    episilon+=episilon_func(S,K,r,q,sigma,T,tipo,quantidade)


AMZN210604C03215000


# Output the Greeks

In [10]:
quantidade_total=0
custo_total=0
for a in range(len(quantidade_list)):
  quantidade_total+=quantidade_list[a]
for b in range(len(custo)):
  custo_total+=custo[b]
print(f"Custo Total {custo_total.round(2)}$")
print("-="*20)
print("First Order Greeks")
print(f"Delta: {delta.round(3)} $/ΔSpot")
print(f"Theta: {theta.round(3)} $/ΔTime")
print(f"Vega: {vega.round(3)} $/ΔVolatility")
print(f"Rho: {rho.round(3)} $/ΔRisk Free rate")
print(f"Episilon: {(episilon).round(3)} $/ΔRisk Free rate")
print("-="*20)
print("Second Order Greeks")
print(f"Gamma: {(gamma).round(3)} ΔDelta/ΔSpot")
print(f"Charm: {(charm).round(3)} ΔDelta/ΔTime")
print(f"Vomma: {(vomma).round(3)} ΔVega/ΔVolatility")
print(f"Vanna: {vanna.round(3)} ΔVega/ΔSpot")
print(f"Veta: {(veta).round(3)} ΔVega/ΔTime")
print("-="*20)
print("Third Order Greeks")
print(f"Speed: {(speed).round(4)} ΔGamma/ΔSpot")
print(f"Zomma: {(zomma).round(4)} ΔGamma/ΔVolatility")
print(f"Color: {(color).round(4)} ΔGamma/ΔTime")
print(f"Ultima: {ultima.round(2)} ΔVomma/ΔVolatility")

Custo Total 59.46$
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
First Order Greeks
Delta: 0.074 $/ΔSpot
Theta: 0.146 $/ΔTime
Vega: -0.054 $/ΔVolatility
Rho: 0.025 $/ΔRisk Free rate
Episilon: -0.026 $/ΔRisk Free rate
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Second Order Greeks
Gamma: -0.0 ΔDelta/ΔSpot
Charm: 0.009 ΔDelta/ΔTime
Vomma: 0.552 ΔVega/ΔVolatility
Vanna: -0.297 ΔVega/ΔSpot
Veta: 0.005 ΔVega/ΔTime
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Third Order Greeks
Speed: -0.0 ΔGamma/ΔSpot
Zomma: 0.0024 ΔGamma/ΔVolatility
Color: 0.0 ΔGamma/ΔTime
Ultima: -6.85 ΔVomma/ΔVolatility
